## Generate gauge for provided URL

## Make table showing comments that cause most discussion

Generate a list of strings showing discussed articles

- What makes a article most discussed
    no. of comments
- top 5 stories with most comments
- time: 

- When provided a URL
- Show most discussed comments for a given article
- select top comments of article, and show kids.

In [ ]:
def get_story_id_from_url(url: str):
    pass

In [ ]:
BASE_URL = "https://hacker-news.firebaseio.com/v0/item/"
def get_comment_ids(story: int) -> list:
    """Returns the id of comments from given story on Hacker News."""
    story_info = get(BASE_URL + f"{story}.json", timeout=30).json()
    comment_ids = story_info.get("kids")
    return comment_ids

In [ ]:
list_kids_num = []
    for comment in comments[0:5]:
        comment_info = get(BASE_URL + f"{comment}.json", timeout=30).json()
        list_kids_num.append({'parent_id': comment, 'num_of_child': len(comment_info.get("kids"))})
        kids = len(comment_info.get("kids"))
        print(f"{comment} has {kids} replies.")

In [41]:
import html
import pprint

def get_top_5_most_replied_parent_comments(story_id: int):
    # get all its parent comments
    parent_comments = get_comment_ids(story_id)
    
    parent_comments_list = []
    for parent_comment_id in parent_comments:
        # figure out a comment's title
        comment_info = get(BASE_URL + f"{parent_comment_id}.json", timeout=30).json()
        
        # Check if "text" key exists in comment_info
        comment_text = comment_info.get("text")
        comment_title = html.unescape(comment_text[:75]) if comment_text else "No title available"
        
        # figure out the number of children it has
        number_of_children = 0
        kids_info = comment_info.get("kids")
        if kids_info is not None:
            number_of_children = len(kids_info)

        # list of dicts where each dictionary is: {parent_comment_id : number_of_children}
        parent_comments_list.append({'title': comment_title, 'number_of_children': number_of_children})

    # sort list of dicts by number_of_children 
    sorted_list = sorted(parent_comments_list, key=lambda comment_dict: comment_dict.get('number_of_children', 0), reverse=True)
    return sorted_list[:5]

pprint.pprint(get_top_5_most_replied_parent_comments(38865518))


[{'number_of_children': 8,
  'title': '> efficiently converts optical power to electrical power<p>Damn, I '
           'thoug'},
 {'number_of_children': 5,
  'title': 'So... I have a really interesting anecdote on <i>"Power over '
           'fiber&quo'},
 {'number_of_children': 4,
  'title': 'From a reply:  “wow. I expected expensive (two digits), but '
           'indeed: these a'},
 {'number_of_children': 4,
  'title': 'How much optical power could be safely carried over such an '
           'optical fiber i'},
 {'number_of_children': 4,
  'title': "What's the use-case here? That's the biggest optoisolator I'"}]


In [42]:
top_5_comments_info = get_top_5_most_replied_parent_comments(38865518)

for parent_comment in top_5_comments_info:
    print(f"{parent_comment.get('title')} [{parent_comment.get('number_of_children')} replies]")
        

> efficiently converts optical power to electrical power<p>Damn, I thoug [8 replies]
So... I have a really interesting anecdote on <i>"Power over fiber&quo [5 replies]
From a reply:  “wow. I expected expensive (two digits), but indeed: these a [4 replies]
How much optical power could be safely carried over such an optical fiber i [4 replies]
What's the use-case here? That's the biggest optoisolator I' [4 replies]


![Alt text](image-1.png)

"Comment" + score + replies

In [ ]:
def load_last_two_records(input_story_id: str) -> pd.DataFrame:
    """Loads records from the last 2 hours for a chosen story.
    Returns them as a Dataframe object."""
    query = f"""
        SELECT
            records.story_id,
            MAX(records.comments) - MIN(records.comments) AS comments_count_change,
            stories.title,
            records.record_time
            FROM records
            JOIN stories ON records.story_id = stories.story_id
            WHERE records.story_id = {int(input_story_id)} 
            GROUP BY records.story_id, stories.title, records.record_time
            ORDER BY records.record_time
            DESC LIMIT 2
        ;
        """
    return pd.read_sql(query, con=get_db_connection())